In [8]:

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,4"

import sys
import torch
import pandas as pd
import numpy as np
import pickle


sys.path.append("../NLP-DL-Project-hypo-to-hyper/pipeline_src/")

from metrics.metrics import get_all_metrics, get_hypernyms
from dataset.dataset import HypernymDataset
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    "databricks/dolly-v2-3b",
    padding_side="left",
)
test_data_path = 'babel_datasets/test_en_babel.pickle'
test_data_path = 'babel_datasets/train_en_babel.pickle'


In [10]:
test_dataset = HypernymDataset(
    data_path=test_data_path,
    tokenizer=tokenizer,
    semeval_format=False,
)

In [7]:
for i in range(len(test_dataset)):
    try:
        test_dataset[i]
    except TypeError:
        print(i)

In [11]:
test_path = "babel_datasets/test_en_babel.pickle"
saving_path = "/raid/rabikov/model_outputs/" + "databricks-dolly-v2-3b_1"

In [12]:
df = pd.read_pickle(test_path)

In [5]:
df

[{'children': 'professional_baseball.n.1',
  'parents': ['baseball.n.1', 'sport.n.2'],
  'grandparents': None,
  'case': 'simple_triplet_2parent'},
 {'children': ['one-hitter.n.1',
   'two-hitter.n.1',
   'three-hitter.n.1',
   'four-hitter.n.1',
   'five-hitter.n.1',
   'hardball.n.2',
   'no-hit_game.n.1',
   'perfect_game.n.1',
   'professional_baseball.n.1',
   'rounders.n.1',
   'softball.n.2',
   'steal.n.2',
   'stickball.n.1'],
  'parents': 'baseball.n.1',
  'grandparents': None,
  'case': 'leafs_and_no_leafs'},
 {'children': '5-methoxytryptamine.n.1',
  'parents': 'serotonin.n.1',
  'grandparents': 'monoamine_neurotransmitter.n.1',
  'case': 'only_child_leaf'},
 {'children': ['quarter_mile.n.1',
   'half_mile.n.1',
   'archine.n.1',
   'body_length.n.1',
   'cable.n.4',
   'cicero.n.1',
   'kos.n.1',
   'cubit.n.1',
   'finger.n.2',
   'em.n.2',
   'en.n.1',
   'fathom.n.1',
   'fistmele.n.1',
   'foot.n.2',
   'footer.n.1',
   'furlong.n.1',
   'geometric_pace.n.1',
   'head.

In [56]:
all_labels = df["hypernym"].apply(lambda x: ",".join(x.split("\t"))).values

In [13]:
with open(saving_path, "rb") as fp:
    all_preds = pickle.load(fp)

In [35]:
df[19], all_preds[19]

({'children': ['Brazilian_rosewood.n.1'],
  'parents': 'rosewood.n.2',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['Indian_blackwood.n.1', 'Honduras_rosewood.n.1']},
 'Brazilian rosewood,')

In [25]:
def delete_techniqal(elem):

    if isinstance(elem, str):
        if '.n.' in elem:
            return elem.split('.')[0].replace('_', ' ')
        
    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word))
        return new_words

In [53]:
df[17]

{'children': ['black_locust.n.2', 'clammy_locust.n.1'],
 'parents': 'locust_tree.n.1',
 'grandparents': None,
 'case': 'only_leafs_divided',
 'brothers': ['courbaril.n.1', 'water_locust.n.1', 'honey_locust.n.1']}

In [55]:
delete_techniqal(df[17]['brothers'])

['courbaril', 'water locust', 'honey locust']

In [58]:
all_pre

1484

In [61]:
metrics = get_all_metrics(all_labels, all_preds, limit=15)

In [62]:
metrics

{'MRR': 0.39,
 'MAP': 0.1479675563942224,
 'P@1': 0.39,
 'P@3': 0.15655555555555697,
 'P@5': 0.1282333333333323,
 'P@15': 0.1189246808746804}

In [44]:
all_labels[4]

'thrower,baseball player,jock,person'

In [43]:
all_preds[4]

'thrower, baseball player, jock, person'